# Working with 3D city models in Python



**Balázs Dukai** [*@BalazsDukai*](https://twitter.com/balazsdukai), **GeoPython 2019**

Tweet <span style="color:blue">#CityJSON</span>

[3D geoinformation research group, TU Delft, Netherlands](https://3d.bk.tudelft.nl/)

![](figures/tud-3dgeoinfo-black.png)

Repo of this talk: [https://github.com/balazsdukai/geopython2019](https://github.com/balazsdukai/geopython2019)

# Why 3D city models?

![3d city model applications](figures/3d_cm_applications.png)

# ...mostly just production of the models

many available, but who **uses** them? **For more than visualisation?**

![open 3d city models](figures/open_cms.png)

# Software are lagging behind

+ not many software supports 3D city models

+ if they do, mostly propietary data model and format

+ large, *"eterprise"*-type applications (think Esri, FME, Bentley ... )

+ few tools accessible for the individual developer / hobbyist

<span style="color:red">Participate in our benchmark:</span> https://3d.bk.tudelft.nl/projects/geobim-benchmark/

# In truth, they are a bit difficult to work with

1. our built environment is complex

2. GML doesn't help ( *[GML madness](http://erouault.blogspot.com/2014/04/gml-madness.html) by Even Rouault* )

![citygml logo](figures/citygml.png)

## The CityGML data model

![](figures/citygml_uml.gif)

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I just got sent a CityGML file. <a href="https://t.co/jnTVoRnVLS">pic.twitter.com/jnTVoRnVLS</a></p>&mdash; James Fee (@jamesmfee) <a href="https://twitter.com/jamesmfee/status/748270105319006208?ref_src=twsrc%5Etfw">June 29, 2016</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 

+ files are deeply nested, and large
+ many "points of entry"
+ many diff ways to do one thing (GML doesn't help, *[GML madness](http://erouault.blogspot.com/2014/04/gml-madness.html) by Even Rouault* )

--> little software support 

![](figures/citygml_encoding.png)

![cityjson logo](figures/cityjson_webpage.png)

## CiyJSON aims to be 

+ *simple*, as in easy to implement
+ designed with programmers in mind
+ fully developed in the open
+ flattened hierarchy
+ implementation first

![GitHub Issues](figures/github_issues.png)

![cityjson paper](figures/cityjson_paper.png)

## Compression ~6x over CityGML

![](figures/zurich_size.png)

## Compression
| file     | CityGML size (original) | CityGML size (w/o spaces) | textures | CityJSON | compression |
| -------- | ----------------------- | ----------------------------- |--------- | ------------ | --------------- | 
| [CityGML demo "GeoRes"](https://www.citygml.org/samplefiles/) | 4.3MB | 4.1MB | yes | 524KB | 8.0 |
| [CityGML v2 demo "Railway"](https://www.citygml.org/samplefiles/) | 45MB | 34MB | yes | 4.3MB | 8.1 |
| [Den Haag "tile 01"](https://data.overheid.nl/data/dataset/ngr-3d-model-den-haag) | 23MB | 18MB | no, material | 2.9MB | 6.2 |
| [Montréal VM05](http://donnees.ville.montreal.qc.ca/dataset/maquette-numerique-batiments-citygml-lod2-avec-textures/resource/36047113-aa19-4462-854a-cdcd6281a5af) | 56MB | 42MB | yes | 5.4MB | 7.8 |
| [New York LoD2 (DA13)](https://www1.nyc.gov/site/doitt/initiatives/3d-building.page) | 590MB | 574MB | no | 105MB | 5.5 |
| [Rotterdam Delfshaven](http://rotterdamopendata.nl/dataset/rotterdam-3d-bestanden/resource/edacea54-76ce-41c7-a0cc-2ebe5750ac18) | 16MB | 15MB | yes | 2.6MB | 5.8 |
| [Vienna (the demo file)](https://www.data.gv.at/katalog/dataset/86d88cae-ad97-4476-bae5-73488a12776d) | 37MB | 36MB | no |  5.3MB | 6.8 |
| [Zürich LoD2](https://www.data.gv.at/katalog/dataset/86d88cae-ad97-4476-bae5-73488a12776d) | 3.03GB | 2.07GB | no |  292MB | 7.1 |

![standards](figures/standards.png)

[https://xkcd.com/927/](https://xkcd.com/927/)

## An empty CityJSON
```json
{
  "type": "CityJSON",
  "version": "1.0",
  "extensions": {},
  "metadata": {},
  "transform": {
    "scale": [],
    "translate": []
  },
    
  "CityObjects": {},          
  "vertices": [],             
    
  "appearance": {},
  "geometry-templates": {}
}
```

## A CityObject

```json
{
    "CityObjects": {
        
      "id-1": {
        "type": "Building",
        "geographicalExtent": [ 84710.1, 446846.0, -5.3, 84757.1, 446944.0, 40.9 ], 
        "attributes": { 
          "measuredHeight": 22.3,
          "roofType": "gable",
          "owner": "Elvis Presley"
        },
          
        "children": ["id-2"],        
          
        "geometry": [{   }]          
      }
    }
    
}
```

## Geometry

Inspired by Wavefront OBJ

We have a vertex list at the root of the document

```json
{
    
"vertices": [
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [8523.134, 487625.134, 2.03]
]
    
}
```

The geometry definition uses vertex indices

First vertex is not repeated (unlike in Simple Features)

```json
{
  "type": "MultiSurface",
  "lod": 2,
  "boundaries": [
    [[0, 3, 2, 1]], [[4, 5, 6, 7]], [[0, 1, 5, 4]]
  ]
}
```

This `MulitSurface` has 

3 surfaces 
```json
[[0, 3, 2, 1]], [[4, 5, 6, 7]], [[0, 1, 5, 4]]
```
each surface has only an exterior ring (the first array)
```json
[ [0, 3, 2, 1] ]
```

### Geometry semantics

```json
{
  "type": "MultiSurface",
  "lod": 2,
    
  "boundaries": [
    [[0, 3, 2, 1]], [[4, 5, 6, 7]], [[0, 1, 5, 4]], [[0, 2, 3, 8]], [[10, 12, 23, 48]]
  ],
    
    
  "semantics": {
    "surfaces" : [
      {
        "type": "WallSurface",
        "slope": 33.4,
        "children": [2]
      }, 
      {
        "type": "RoofSurface",
        "slope": 66.6
      }
    ],
    "values": [0, 0, null, 1, 2]
  }
    
}
```

In [1]:
import json

path = './data/rotterdam_subset.json'
with open(path) as fin:
    cm = json.loads(fin.read())
    
print("There are %s CityObjects" % len(cm['CityObjects']))

# list all IDs
for id in cm['CityObjects']:
    print(id, "\t")

There are 16 CityObjects
{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE} 	
{71B60053-BC28-404D-BAB9-8A642AAC0CF4} 	
{6271F75F-E8D8-4EE4-AC46-9DB02771A031} 	
{DE77E78F-B110-43D2-A55C-8B61911192DE} 	
{19935DFC-F7B3-4D6E-92DD-C48EE1D1519A} 	
{953BC999-2F92-4B38-95CF-218F7E05AFA9} 	
{8D716FDE-18DD-4FB5-AB06-9D207377240E} 	
{C6AAF95B-8C09-4130-AB4D-6777A2A18A2E} 	
{72390BDE-903C-4C8C-8A3F-2DF5647CD9B4} 	
{8244B286-63E2-436E-9D4E-169B8ACFE9D0} 	
{87316D28-7574-4763-B9CE-BF6A2DF8092C} 	
{CD98680D-A8DD-4106-A18E-15EE2A908D75} 	
{64A9018E-4F56-47CD-941F-43F6F0C4285B} 	
{459F183A-D0C2-4F8A-8B5F-C498EFDE366D} 	
{237D41CC-991E-4308-8986-42ABFB4F7431} 	
{23D8CA22-0C82-4453-A11E-B3F2B3116DB4} 	


+ Working with a CityJSON file is straigforward. One can open it with the standard library and get going.
+ But you need to know the schema well.
+ And you need to write everything from scratch.

![cjio](figures/cjio_docs.png)

That is why we are developing **cjio**. 

**cjio** is how *we eat what we cook*

Aims to help to actually work with and analyse 3D city models, and extract more value from them. Instead of letting them gather dust in some governmental repository.

## `cjio` has a (quite) stable CLI

```bash
$ cjio city_model.json reproject 2056 export --format glb /out/model.glb
```

## and an experimental API

```python
from cjio import cityjson

cm = cityjson.load('city_model.json')

cm.get_cityobjects(type='building')
```

**`pip install cjio`**

**`pip install git+https://github.com/tudelft3d/cjio@develop`**

# `cjio`'s CLI

In [2]:
! cjio --help

Usage: cjio [OPTIONS] INPUT COMMAND1 [ARGS]... [COMMAND2 [ARGS]...]...

  Process and manipulate a CityJSON file, and allow different outputs. The
  different operators can be chained to perform several processing in one
  step, the CityJSON model goes through the different operators.

  To get help on specific command, eg for 'validate':

      cjio validate --help

  Usage examples:

      cjio example.json info validate
      cjio example.json assign_epsg 7145 remove_textures export output.obj
      cjio example.json subset --id house12 save out.json

Options:
  --version                Show the version and exit.
  --ignore_duplicate_keys  Load a CityJSON file even if some City Objects have
                           the same IDs (technically invalid file)
  --help                   Show this message and exit.

Commands:
  assign_epsg                Assign a (new) EPSG.
  clean                      Clean = remove_duplicate_vertices +...
  compress                   Compress a CityJS

In [3]:
! cjio data/rotterdam_subset.json info

Parsing data/rotterdam_subset.json
{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "cityobjects_total": 16,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 383,
  "transform/compressed": true,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "level_of_detail": [
    2
  ],
  "semantics_surfaces_present": [
    "GroundSurface",
    "RoofSurface",
    "WallSurface"
  ],
  "cityobject_attributes": [
    "status",
    "bron_tex",
    "TerrainHeight",
    "bron_geo",
    "voll_tex"
  ],
  "materials": false,
  "textures": true
}


In [4]:
! cjio data/rotterdam_subset.json validate

Parsing data/rotterdam_subset.json
===== Validation (with official CityJSON schemas) =====
-- Validating the syntax of the file
	(using the schemas 1.0.0)
-- Validating the internal consistency of the file (see docs for list)
	--Vertex indices coherent
	--Specific for CityGroups
	--Semantic arrays coherent with geometry
	--Root properties
	--Empty geometries
	--Duplicate vertices
	--Orphan vertices
	--CityGML attributes
=====
File is valid
File has warnings
--- WARNINGS ---
	(16 CityObjects have this warning)
	(16 CityObjects have this warning)
	(16 CityObjects have this warning)
	(16 CityObjects have this warning)
	(16 CityObjects have this warning)


In [5]:
! cjio data/rotterdam_subset.json \
    subset --exclude --id "{CD98680D-A8DD-4106-A18E-15EE2A908D75}" \
    merge data/rotterdam_one.json \
    reproject 2056 \
    save data/test_rotterdam.json

Parsing data/rotterdam_subset.json
Subset of CityJSON
Merging files
	{
  "cityjson_version": "1.0",
  "epsg": 28992,
  "cityobjects_total": 1,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 25,
  "transform/compressed": true,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "level_of_detail": [
    2
  ],
  "semantics_surfaces_present": [
    "WallSurface",
    "GroundSurface",
    "RoofSurface"
  ],
  "cityobject_attributes": [
    "TerrainHeight",
    "status",
    "voll_tex",
    "bron_geo",
    "bron_tex"
  ],
  "materials": false,
  "textures": true
}
Reproject to EPSG:2056
  [####################################]  100%          
Saving CityJSON to a file /home/balazs/Reports/talk_cjio_geopython_2019/data/test_rotterdam.json


```python
class CityJSON:
    # One class does everything
    
    def remove_materials(self):
        for i in self.j["CityObjects"]:
            if "material" in self.j["CityObjects"][i]:
                del self.j["CityObjects"][i]["material"]
        if "appearance" in self.j:
            if "materials" in self.j["appearance"]:
                del self.j["appearance"]["materials"]
            if "default-theme-material" in self.j["appearance"]:
                del self.j["appearance"]["default-theme-material"]
        if self.j["appearance"] is None or len(self.j["appearance"]) == 0:
            del self.j["appearance"]
        return True
```

+ The CLI was first, no plans for API
+ Function for the CLI work with the JSON directly, passing it along
+ Simple and effective architecture


+ Works with whole city model only
+ Sensitive to changes in the schema
+ Not very "pythonic" (I think...)

# `cjio`'s API

Allow    *read* --> *explore* --> *modify* --> *write*    iteration

Work with CityObjects and their parts

Functions for common operations

Inspired by the *tidyverse* from the R ecosystem

In [6]:
import os
from copy import deepcopy
from cjio import cityjson

This is a subset of the 3D city model of Rotterdam
![](figures/rotterdam_subset.png)

The `load()` method loads a CityJSON file into a CityJSON object.

In [7]:
path = os.path.join('data', 'rotterdam_subset.json')

cm = cityjson.load(path)

print(type(cm))

<class 'cjio.cityjson.CityJSON'>


## Using the CLI commands in the API
You can use any of the CLI commands on a CityJSON object 

*However,* not all CLI commands are mapped 1-to-1 to `CityJSON` methods

And we haven't harmonized the CLI and the API yet. 

In [8]:
cm.validate()

-- Validating the syntax of the file
	(using the schemas 1.0.0)
-- Validating the internal consistency of the file (see docs for list)
	--Vertex indices coherent
	--Specific for CityGroups
	--Semantic arrays coherent with geometry
	--Root properties
	--Empty geometries
	--Duplicate vertices
	--Orphan vertices
	--CityGML attributes


(True,
 False,
 [],
 ["WARNING: attributes 'TerrainHeight' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'voll_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_geo' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'status' not in CityGML schema",
  '\t(16 CityObjects have this warning)'])

## Explore the city model

Print the basic information about the city model. Note that `print()` returns the same information as the `info` command in the CLI.

In [9]:
print(cm)

{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "cityobjects_total": 16,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 383,
  "transform/compressed": true,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "level_of_detail": [
    2
  ],
  "semantics_surfaces_present": [
    "GroundSurface",
    "WallSurface",
    "RoofSurface"
  ],
  "cityobject_attributes": [
    "voll_tex",
    "status",
    "bron_tex",
    "bron_geo",
    "TerrainHeight"
  ],
  "materials": false,
  "textures": true
}


## Getting objects from the model
Get CityObjects by their *type*, or a list of types. Also by their IDs. 

Note that `get_cityobjects()` == `cm.cityobjects`

In [10]:
buildings = cm.get_cityobjects(type='building')

# both Building and BuildingPart objects
buildings_parts = cm.get_cityobjects(type=['building', 'buildingpart'])

r_ids = ['{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}',
         '{6271F75F-E8D8-4EE4-AC46-9DB02771A031}']
buildings_ids = cm.get_cityobjects(id=r_ids)

## Properties and geometry of objects

In [11]:
b01 = buildings_ids['{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}']
print(b01)

{
  "id": "{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}",
  "type": "Building",
  "attributes": {
    "TerrainHeight": 3.03,
    "bron_tex": "UltraCAM-X 10cm juni 2008",
    "voll_tex": "complete",
    "bron_geo": "Lidar 15-30 punten - nov. 2008",
    "status": "1"
  },
  "children": null,
  "parents": null,
  "geometry_type": [
    "MultiSurface"
  ],
  "geometry_lod": [
    2
  ],
  "semantic_surfaces": [
    "GroundSurface",
    "WallSurface",
    "RoofSurface"
  ]
}


In [12]:
b01.attributes

{'TerrainHeight': 3.03,
 'bron_tex': 'UltraCAM-X 10cm juni 2008',
 'voll_tex': 'complete',
 'bron_geo': 'Lidar 15-30 punten - nov. 2008',
 'status': '1'}

CityObjects can have *children* and *parents*

In [13]:
b01.children is None and b01.parents is None

True

CityObject geometry is a list of `Geometry` objects. That is because a CityObject can have multiple geometry representations in different levels of detail, eg. a geometry in LoD1 and a second geometry in LoD2.

In [14]:
b01.geometry

In [15]:
geom = b01.geometry[0]
print("{}, lod {}".format(geom.type, geom.lod))

MultiSurface, lod 2


### Geometry boundaries and Semantic Surfaces
On the contrary to a CityJSON file, the geometry boundaries are dereferenced when working with the API. This means that the vertex coordinates are included in the boundary definition, not only the vertex indices.

`cjio` doesn't provide specific geometry classes (yet), eg. MultiSurface or Solid class. If you are working with the geometry boundaries, you need to the geometric operations yourself, or cast the boundary to a geometry-class of some other library. For example `shapely` if 2D is enough.

In [17]:
transformation_object = cm.transform

# Coordinates are kept 'as is' on load, often compressed. Transform if needed.
geom_transformed = geom.transform(transformation_object)

# get the first surface
geom_transformed.boundaries[0][0]

[(90988.79100000001, 435638.657, 10.652000000000001),
 (90987.429, 435642.77, 10.652000000000001),
 (90986.46900000001, 435641.09, 10.652000000000001),
 (90985.781, 435640.846, 10.652000000000001),
 (90986.801, 435637.955, 10.652000000000001)]

In [18]:
cm_transformed = cityjson.load(path, transform=True)
print(cm_transformed)

{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "cityobjects_total": 16,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 383,
  "transform/compressed": false,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "level_of_detail": [
    2
  ],
  "semantics_surfaces_present": [
    "GroundSurface",
    "WallSurface",
    "RoofSurface"
  ],
  "cityobject_attributes": [
    "voll_tex",
    "status",
    "bron_tex",
    "bron_geo",
    "TerrainHeight"
  ],
  "materials": false,
  "textures": true
}


Semantic Surfaces are stored in a similar fashion as in a CityJSON file, in the `surfaces` attribute of a Geometry object.

In [19]:
geom.surfaces

{0: {'surface_idx': [[0], [1], [2]], 'type': 'RoofSurface'},
 1: {'surface_idx': [[3]], 'type': 'GroundSurface'},
 2: {'surface_idx': [[4],
   [5],
   [6],
   [7],
   [8],
   [9],
   [10],
   [11],
   [12],
   [13],
   [14],
   [15],
   [16],
   [17],
   [18],
   [19]],
  'type': 'WallSurface'}}

`surfaces` does not store geometry boundaries, just references (`surface_idx`). Use the `get_surface_boundaries()` to extract the boundary.

In [20]:
roofs = geom.get_surfaces(type='roofsurface')
roofs

{0: {'surface_idx': [[0], [1], [2]], 'type': 'RoofSurface'}}

In [ ]:
roof_boundaries = []
for r in roofs.values():
    roof_boundaries.append(geom.get_surface_boundaries(r))

In [ ]:
roof_boundaries

### Assigning attributes to Semantic Surfaces
1. extract the surfaces,
2. make the changes on the surface,
3. overwrite the CityObjects with the changes.

In [22]:
cm_copy = deepcopy(cm)
new_cos = {}
for co_id, co in cm.cityobjects.items():
    new_geoms = []
    for geom in co.geometry:
        # Only LoD >= 2 models have semantic surfaces
        if geom.lod >= 2.0:
            # Extract the surfaces
            roofsurfaces = geom.get_surfaces('roofsurface')
            for i, rsrf in roofsurfaces.items():
                # Change the attributes
                if 'attributes' in rsrf.keys():
                    rsrf['attributes']['cladding'] = 'tiles'
                else:
                    rsrf['attributes'] = {}
                    rsrf['attributes']['cladding'] = 'tiles'
                geom.surfaces[i] = rsrf
            new_geoms.append(geom)
        else:
            # Use the unchanged geometry
            new_geoms.append(geom)
    co.geometry = new_geoms
    new_cos[co_id] = co
cm_copy.cityobjects = new_cos

In [23]:
print(cm_copy.cityobjects['{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}'])

{
  "id": "{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}",
  "type": "Building",
  "attributes": {
    "TerrainHeight": 3.03,
    "bron_tex": "UltraCAM-X 10cm juni 2008",
    "voll_tex": "complete",
    "bron_geo": "Lidar 15-30 punten - nov. 2008",
    "status": "1"
  },
  "children": null,
  "parents": null,
  "geometry_type": [
    "MultiSurface"
  ],
  "geometry_lod": [
    2
  ],
  "semantic_surfaces": [
    "GroundSurface",
    "WallSurface",
    "RoofSurface"
  ]
}


### Create new Semantic Surfaces
The process is similar as previously. However, in this example we create new SemanticSurfaces that hold the values which we compute from the geometry. The input city model has a single semantic "WallSurface", without attributes, for all the walls of a building. The snippet below illustrates how to separate surfaces and assign the semantics to them.

In [24]:
new_cos = {}

for co_id, co in cm_copy.cityobjects.items():
    new_geoms = []
    
    for geom in co.geometry:
        if geom.lod >= 2.0:
            max_id = max(geom.surfaces.keys())
            old_ids = []
            
            for w_i, wsrf in geom.get_surfaces('wallsurface').items():
                old_ids.append(w_i)
                del geom.surfaces[w_i]
                boundaries = geom.get_surface_boundaries(wsrf)
                
                for j, boundary_geometry in enumerate(boundaries):
                    # The original geometry has the same Semantic for all wall, 
                    # but we want to divide the wall surfaces by their orientation, 
                    # thus we need to have the correct surface index
                    surface_index = wsrf['surface_idx'][j]
                    new_srf = {
                        'type': wsrf['type'],
                        'surface_idx': surface_index
                    }
                    
                    for multisurface in boundary_geometry:
                        # Do any operation here
                        x, y, z = multisurface[0]
                        if j % 2 > 0:
                            orientation = 'north'
                        else:
                            orientation = 'south'
                        
                        # Add the new attribute to the surface 
                        if 'attributes' in wsrf.keys():
                            wsrf['attributes']['orientation'] = orientation
                        else:
                            wsrf['attributes'] = {}
                            wsrf['attributes']['orientation'] = orientation
                        
                        new_srf['attributes'] = wsrf['attributes']
                        
                        # if w_i in geom.surfaces.keys():
                        #     del geom.surfaces[w_i]
                        
                        max_id = max_id + 1
                        geom.surfaces[max_id] = new_srf
                        
            new_geoms.append(geom)
            
        else:
            # If LoD1, just add the geometry unchanged
            new_geoms.append(geom)
            
    co.geometry = new_geoms
    new_cos[co_id] = co
    
cm_copy.cityobjects = new_cos

## Save or Export
At the end, the `save()` method saves the edited city model into a CityJSON file.

In [25]:
path_out = os.path.join('data', 'test_output.json')
cityjson.save(cm, path_out)

It is also possible to export the city model into a pandas DataFrame. Note that only the CityObject attributes are exported into the dataframe, with CityObject IDs as the index of the dataframe. Thus if you want to export the attributes of SemanticSurfaces for example, then you need to add them as CityObject attributes.

In [26]:
new_cos = {}
for co_id, co in cm.cityobjects.items():
    for geom in co.geometry:
        for srf in geom.surfaces.values():
            if 'attributes' in srf:
                for attr,a_v in srf['attributes'].items():
                    if (attr not in co.attributes) or (co.attributes[attr] is None):
                        co.attributes[attr] = [a_v]
                    else:
                        co.attributes[attr].append(a_v)
    new_cos[co_id] = co
cm_copy.cityobjects = new_cos

In [27]:
df = cm_copy.to_dataframe()
df.head()

,TerrainHeight,bron_geo,bron_tex,cladding,orientation,status,voll_tex
{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE},3.03,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,[tiles],"[north, north, north, north, north, north, nor...",1,complete
{71B60053-BC28-404D-BAB9-8A642AAC0CF4},2.68,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,[tiles],"[north, north, north, north, north, north, nor...",1,complete
{6271F75F-E8D8-4EE4-AC46-9DB02771A031},3.12,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,[tiles],"[north, north, north, north, north, north, nor...",1,complete
{DE77E78F-B110-43D2-A55C-8B61911192DE},2.88,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,[tiles],"[north, north, north, north, north, north, nor...",1,complete
{19935DFC-F7B3-4D6E-92DD-C48EE1D1519A},2.82,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,[tiles],"[south, south, south, south, south, south, sou...",1,complete


# Other software

## Online CityJSON viewer

![](figures/viewer.png)

## QGIS plugin
![](figures/qgis_zurich.png)

## Azul
![](figures/azul.png)

# Full conversion CityGML <--> CityJSON
![](figures/citygml4j.png)

# Thank you!

Balázs Dukai

b.dukai@tudelft.nl

@BalazsDukai

## A few links

[cityjson.org](cityjson.org)

[viewer.cityjson.org](viewer.cityjson.org)

QGIS plugin: [github.com/tudelft3d/cityjson-qgis-plugin](github.com/tudelft3d/cityjson-qgis-plugin)

Azul – CityJSON viewer on Mac – check the [AppStore](https://apps.apple.com/nl/app/azul/id1173239678?mt=12)

cjio: [github.com/tudelft3d/cjio](github.com/tudelft3d/cjio) & [tudelft3d.github.io/cjio/](tudelft3d.github.io/cjio/)